# Scenario Discovery Demo: PRIM, PCA–PRIM, and CART

This interactive notebook demonstrates how different scenario discovery algorithms isolate regions of interest in a 2D input space.  
You can explore the behaviour of PRIM, PCA–PRIM, and CART by adjusting dataset and algorithm settings.

All results are saved to:

`_data/scenario_methods_demo_outputs/`

In [1]:
# === Imports and Setup ===
import os
import sys
sys.path.insert(0, os.path.abspath("..")) 

import numpy as np
import ipywidgets as widgets
from IPython.display import display

from notebook_helpers import update_plots, save_prim_plots, save_cart_plots

## Configuration Panel

Use the sliders below to control:

- **Dataset Size**: Total number of generated points
- **Quadrilateral Corners**: Defines the region of interest
- **Noise Settings**: Adjust class noise inside/outside the shape
- **Algorithm Settings**: Configure PRIM and CART behaviour

In [2]:
# Create input widgets
num_dots_slider = widgets.IntSlider(value=1700, min=100, max=2500, step=100, description="Num Dots")
default_quad = np.array([0.3, 0.9, 0.9, 0.7, 1.0, 0.4, 0.2, 0.55])
corner_labels = ["Corner 1 X", "Corner 1 Y", "Corner 2 X", "Corner 2 Y",
                 "Corner 3 X", "Corner 3 Y", "Corner 4 X", "Corner 4 Y"]
quad_sliders = [widgets.FloatSlider(value=val, min=0.0, max=1.0, step=0.05, description=lbl)
                for lbl, val in zip(corner_labels, default_quad)]
frac_inside_slider = widgets.FloatSlider(value=0.95, min=0.7, max=1.0, step=0.05, description="Frac Inside")
frac_outside_slider = widgets.FloatSlider(value=0.05, min=0.0, max=0.3, step=0.05, description="Frac Outside")
peel_frac_slider = widgets.FloatSlider(value=0.10, min=0.0, max=0.5, step=0.05, description="Peel Frac")
prim_mass_min_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description="PRIM Mass Min")
cart_mass_min_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description="CART Mass Min")
save_prim_button = widgets.Button(description="Save PRIM Plots")
save_cart_button = widgets.Button(description="Save CART Plots")

## Output Grid

This tool generates nine visual outputs in a 3×3 layout:

1. Sampled data with class labels  
2. PRIM box evolution over iterations  
3. PRIM peeling trajectory  
4. PCA–rotated input space  
5. PCA–PRIM box evolution (in rotated space)  
6. PCA–PRIM box evolution (in original space)  
7. PCA–PRIM peeling trajectory  
8. PRIM vs. PCA–PRIM trajectory comparison  
9. CART decision regions + coverage table

In [3]:
# Create output areas
plot_outputs = [widgets.Output(layout=widgets.Layout(width="100%", height="300px"))
                for _ in range(9)]
table_output = widgets.Output(layout=widgets.Layout(width="100%", height="150px"))
grid = widgets.GridspecLayout(3, 3, width="12in", height="9in")
for i in range(3):
    for j in range(3):
        grid[i, j] = plot_outputs[i * 3 + j]

## Plot Update Logic

This function runs when any slider is changed. It updates all plot panels and regenerates the decision tree table.

In [4]:
def on_update(_):
    update_plots(
        quad_sliders,
        num_dots_slider.value,
        frac_inside_slider.value,
        frac_outside_slider.value,
        peel_frac_slider.value,
        prim_mass_min_slider.value,
        cart_mass_min_slider.value,
        plot_outputs,
        table_output,
    )

## Set Up Observers and Buttons

Connect each slider to the `on_update` function and initialise the layout.

In [5]:
# Attach observers
num_dots_slider.observe(on_update, names="value")
frac_inside_slider.observe(on_update, names="value")
frac_outside_slider.observe(on_update, names="value")
peel_frac_slider.observe(on_update, names="value")
prim_mass_min_slider.observe(on_update, names="value")
cart_mass_min_slider.observe(on_update, names="value")
for slider in quad_sliders:
    slider.observe(on_update, names="value")

# Initial update
on_update(None)

## Save Buttons

These buttons export the currently generated plots as PDFs to the output folder.

In [6]:
save_prim_button.on_click(lambda b: save_prim_plots(
    quad_sliders,
    num_dots_slider.value,
    frac_inside_slider.value,
    frac_outside_slider.value,
    peel_frac_slider.value,
    prim_mass_min_slider.value,
    cart_mass_min_slider.value,
))
save_cart_button.on_click(lambda b: save_cart_plots(
    quad_sliders,
    num_dots_slider.value,
    frac_inside_slider.value,
    frac_outside_slider.value,
    peel_frac_slider.value,
    prim_mass_min_slider.value,
    cart_mass_min_slider.value
))

## Display Full Interface

Configure the plots using the sliders and when finished click **Save PRIM Plots** or **Save CART Plots** to export results as PDFs.  
Files are written under their respective folders inside:

`_data/scenario_methods_demo_outputs/`

In [7]:
input_widgets = widgets.VBox([
    num_dots_slider,
    widgets.HBox([frac_inside_slider, frac_outside_slider, peel_frac_slider]),
    widgets.HBox(quad_sliders[:4]),
    widgets.HBox(quad_sliders[4:]),
    widgets.HBox([prim_mass_min_slider, cart_mass_min_slider]),
    save_prim_button,
    save_cart_button,
])
display(input_widgets, grid, table_output)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001', height='300px', width='100%')), Output(la…

Output(layout=Layout(height='585px', width='100%'))